In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.models import Model
from keras.utils import plot_model

np.set_printoptions(2)

Using TensorFlow backend.


## Load data

In [14]:
sequence = np.arange(50) / 50
n_features = 5
n_in = len(sequence) // n_features
sequence = sequence.reshape((1, n_in, n_features))

## Model

In [15]:
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_in,n_features)))
model.add(RepeatVector(n_in))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 100)               42400     
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 10, 100)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 10, 100)           80400     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 10, 5)             505       
Total params: 123,305
Trainable params: 123,305
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
model.fit(sequence, sequence, epochs=300, verbose=0)

In [17]:
yhat = model.predict(sequence, verbose=0)
print(yhat-sequence)

[[[ 2.15e-02  1.55e-02  1.88e-02  1.94e-02  3.55e-03]
  [-7.94e-03 -6.23e-03 -1.30e-03  5.87e-03 -4.43e-03]
  [-1.18e-02 -8.43e-03 -5.64e-03  5.82e-04 -3.88e-03]
  [-5.17e-03 -2.65e-03 -2.84e-03 -7.26e-04 -8.61e-04]
  [ 3.90e-03  4.05e-03  1.01e-03 -2.05e-03  3.39e-03]
  [ 9.73e-03  7.70e-03  2.99e-03 -4.01e-03  5.79e-03]
  [ 6.63e-03  4.36e-03 -4.42e-05 -6.15e-03  2.75e-03]
  [ 7.88e-04 -6.78e-04 -3.10e-03 -5.88e-03 -1.40e-03]
  [-3.81e-03 -3.79e-03 -2.76e-03 -1.04e-03 -3.40e-03]
  [-4.17e-03 -2.09e-03  3.86e-03  1.04e-02 -5.00e-04]]]


## Keep Standalone LSTM Encoder

In [6]:
# connect the encoder LSTM as the output layer
encoder_model = Model(inputs=model.inputs, outputs=model.layers[0].output)
# plot_model(encoder_model, show_shapes=True, to_file='lstm_encoder.png')
# get the feature vector for the input sequence
yhat_enc = encoder_model.predict(sequence)
print(yhat_enc.shape)
print(yhat_enc)

(1, 100)
[[1.62e-03 2.49e-01 0.00e+00 0.00e+00 9.83e-02 0.00e+00 6.87e-02 0.00e+00
  5.18e-02 0.00e+00 1.99e-02 4.03e-02 0.00e+00 1.49e-01 2.10e-04 0.00e+00
  2.58e-01 0.00e+00 0.00e+00 0.00e+00 0.00e+00 1.32e-01 3.08e-01 0.00e+00
  1.59e-01 0.00e+00 0.00e+00 0.00e+00 1.45e-01 1.33e-01 0.00e+00 4.15e-02
  0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 9.97e-02
  3.00e-01 0.00e+00 0.00e+00 0.00e+00 0.00e+00 1.06e-01 6.37e-02 0.00e+00
  1.22e-01 0.00e+00 4.82e-02 4.94e-02 1.64e-01 2.86e-01 0.00e+00 0.00e+00
  3.32e-02 1.54e-01 0.00e+00 0.00e+00 2.64e-01 6.93e-02 0.00e+00 1.39e-03
  1.26e-01 0.00e+00 2.33e-01 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 1.59e-01 0.00e+00 4.71e-02 2.02e-01 1.74e-01 2.98e-02 0.00e+00
  0.00e+00 0.00e+00 2.84e-01 1.30e-04 5.93e-06 0.00e+00 0.00e+00 1.19e-01
  1.35e-01 0.00e+00 0.00e+00 1.96e-01 2.22e-05 0.00e+00 2.21e-01 1.47e-01
  0.00e+00 0.00e+00 1.65e-01 0.00e+00]]


## Findings

If input range is large, the absolute precision will drop. But relative precision according to the value range does not change. So pruning extreme values is necessary.

## References

1. [Machine Learning Mastery: A Gentle Introduction to LSTM Autoencoders](https://machinelearningmastery.com/lstm-autoencoders/)